In [2]:
import pickle

with open('dataframe.pkl', 'rb') as file:
    df = pickle.load(file)

### On définit la partie Train et Test

In [3]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

In [4]:
train = df.loc[(df['DateTransaction'] >= train_inf) & (df['DateTransaction'] <= train_sup)]
X_train = train.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_train = train['FlagImpaye']

In [5]:
test = df.loc[(df['DateTransaction'] >= test_inf) & (df['DateTransaction'] <= test_sup)]
X_test = test.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_test = test['FlagImpaye']

In [23]:
len(X_train)

3888468

In [24]:
len(X_test)

737068

## Test de chaque méthode d'échantillonage par modeles

In [28]:
from sklearn import metrics
from sklearn.metrics import f1_score

In [27]:
#!pip install 'imblearn'
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE

In [26]:
#!pip install xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


On fait tourner DecisionTreeClassifier, RandomForestClassifier, GradientBoostingClassifier,KNeighborsClassifier, SVC, adaboost et xgboost

In [26]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from tqdm import tqdm

In [11]:
# Définit les stratégies d'échantillonnage
sampling_strategies = [0.05, 0.1]  
#on garde deux oversampling et deux undersampling
samplers = [RandomOverSampler, SMOTE, RandomUnderSampler, NearMiss]

# Précalcule les échantillons resamplés pour chaque méthode d'échantillonnage
resampled_data = {}
for sampler in tqdm(samplers, desc="Resampling Progress"):
    for strategy in tqdm(sampling_strategies, desc=f"{sampler.__name__} Progress", leave=False):
        key = (sampler, strategy)
        X_resampled, y_resampled = sampler(sampling_strategy=strategy).fit_resample(X_train, y_train)
        resampled_data[key] = (X_resampled, y_resampled)

Resampling Progress:   0%|          | 0/4 [00:00<?, ?it/s]

Resampling Progress: 100%|██████████| 4/4 [14:03<00:00, 210.99s/it]


In [ ]:
# Définir les modèles à tester
models = [
    DecisionTreeClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_1 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_1 = pd.concat([results_df_1, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_1)
results_df_1.to_pickle('dataframe_recap_sampling_1.pkl')

In [ ]:

# Définir les modèles à tester
models = [
    RandomForestClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_2 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_2 = pd.concat([results_df_2, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_2)
results_df_2.to_pickle('dataframe_recap_sampling_2.pkl')

In [16]:

# Définir les modèles à tester
models = [
    SVC()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_3 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_3 = pd.concat([results_df_3, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_3)
results_df_3.to_pickle('dataframe_recap_sampling_3.pkl')

In [15]:
# Définir les modèles à tester
models = [
    AdaBoostClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_4 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_4 = pd.concat([results_df_4, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_4)
results_df_4.to_pickle('dataframe_recap_sampling_4.pkl')

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_4044\3750963144.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_4 = pd.concat([results_df_4, pd.DataFrame({


Model: AdaBoostClassifier, Sampler: RandomOverSampler, Strategy: 0.05, F1 Score: 0.11692036179130819


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Model: AdaBoostClassifier, Sampler: RandomOverSampler, Strategy: 0.1, F1 Score: 0.13142780820088154


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


...

Model: AdaBoostClassifier, Sampler: RandomUnderSampler, Strategy: 0.1, F1 Score: 0.12748484650980904


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Model: AdaBoostClassifier, Sampler: NearMiss, Strategy: 0.05, F1 Score: 0.03958164977932492


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Model: AdaBoostClassifier, Sampler: NearMiss, Strategy: 0.1, F1 Score: 0.03174043378592841
                Model             Sampler  Sampling Strategy  F1 Score
0  AdaBoostClassifier   RandomOverSampler               0.05  0.116920
1  AdaBoostClassifier   RandomOverSampler               0.10  0.131428
2  AdaBoostClassifier               SMOTE               0.05  0.015576
3  AdaBoostClassifier               SMOTE               0.10  0.022629
4  AdaBoostClassifier  RandomUnderSampler               0.05  0.107463
5  AdaBoostClassifier  RandomUnderSampler               0.10  0.127485
6  AdaBoostClassifier            NearMiss               0.05  0.039582
7  AdaBoostClassifier            NearMiss               0.10  0.031740


In [14]:
# Définir les modèles à tester
models = [
    XGBClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_5 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_5 = pd.concat([results_df_5, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_5)
results_df_5.to_pickle('dataframe_recap_sampling_5.pkl')

C:\Users\Admin\AppData\Local\Temp\ipykernel_4044\3536015111.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_5 = pd.concat([results_df_5, pd.DataFrame({


Model: XGBClassifier, Sampler: RandomOverSampler, Strategy: 0.05, F1 Score: 0.0715716151501959
Model: XGBClassifier, Sampler: RandomOverSampler, Strategy: 0.1, F1 Score: 0.058997050147492625
Model: XGBClassifier, Sampler: SMOTE, Strategy: 0.05, F1 Score: 0.06164853474468294
Model: XGBClassifier, Sampler: SMOTE, Strategy: 0.1, F1 Score: 0.07784960871044573
Model: XGBClassifier, Sampler: RandomUnderSampler, Strategy: 0.05, F1 Score: 0.07633948494474051
Model: XGBClassifier, Sampler: RandomUnderSampler, Strategy: 0.1, F1 Score: 0.0656522313176657
Model: XGBClassifier, Sampler: NearMiss, Strategy: 0.05, F1 Score: 0.03485663082437276
Model: XGBClassifier, Sampler: NearMiss, Strategy: 0.1, F1 Score: 0.029324231670471584
           Model             Sampler  Sampling Strategy  F1 Score
0  XGBClassifier   RandomOverSampler               0.05  0.071572
1  XGBClassifier   RandomOverSampler               0.10  0.058997
2  XGBClassifier               SMOTE               0.05  0.061649
3  XGBClassi

In [17]:
from sklearn.linear_model import LogisticRegression

# Définir les modèles à tester
models = [
    LogisticRegression(max_iter= 1000) 
]

# Initialiser un DataFrame pour stocker les résultats
results_df_6 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle de régression logistique
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_6 = pd.concat([results_df_6, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_6)
results_df_6.to_pickle('dataframe_recap_sampling_6_1000.pkl')


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Admin\AppData\Local\Temp\ipykernel_24944\664018385.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_6 = pd.concat([r

Model: LogisticRegression, Sampler: RandomOverSampler, Strategy: 0.05, F1 Score: 0.05769485245467778


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: LogisticRegression, Sampler: RandomOverSampler, Strategy: 0.1, F1 Score: 0.10608451692091882


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Model: LogisticRegression, Sampler: RandomUnderSampler, Strategy: 0.1, F1 Score: 0.10295805739514349


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: LogisticRegression, Sampler: NearMiss, Strategy: 0.05, F1 Score: 0.04350021884785024


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: LogisticRegression, Sampler: NearMiss, Strategy: 0.1, F1 Score: 0.03685105463396325
                Model             Sampler  Sampling Strategy  F1 Score
0  LogisticRegression   RandomOverSampler               0.05  0.057695
1  LogisticRegression   RandomOverSampler               0.10  0.106085
2  LogisticRegression               SMOTE               0.05  0.054252
3  LogisticRegression               SMOTE               0.10  0.102430
4  LogisticRegression  RandomUnderSampler               0.05  0.066667
5  LogisticRegression  RandomUnderSampler               0.10  0.102958
6  LogisticRegression            NearMiss               0.05  0.043500
7  LogisticRegression            NearMiss               0.10  0.036851


On réunit les résultats dans un seul tableau recapitulatif

In [24]:
results_df_1 = pd.read_pickle('dataframe_recap_sampling_1.pkl')
results_df_2 = pd.read_pickle('dataframe_recap_sampling_2.pkl')
#results_df_3 = pd.read_pickle('dataframe_recap_sampling_3.pkl')
results_df_4 = pd.read_pickle('dataframe_recap_sampling_4.pkl')
results_df_5 = pd.read_pickle('dataframe_recap_sampling_5.pkl')
results_df_6 = pd.read_pickle('dataframe_recap_sampling_6_1000.pkl')

In [10]:
results= pd.concat([results_df_1, results_df_2, results_df_4, results_df_5, results_df_6], ignore_index=True)
results.to_pickle('dataframe_recap_sampling1.pkl')

In [11]:
result_sampling = pd.read_pickle('dataframe_recap_sampling1.pkl')
result_no_sampling = pd.read_pickle('dataframe_recap_no_sampling.pkl')

In [12]:
results_df = pd.concat([result_sampling, result_no_sampling])
results_df.to_pickle('dataframe_recap_sampling_no_sampling.pkl')

In [13]:
results_df.reset_index(drop=True, inplace=True)

In [15]:
results_df.describe(include='all')

,Model,Sampler,Sampling Strategy,F1 Score
count,49,40,40.000000,49.000000
unique,5,4,NaN,NaN
top,RandomForestClassifier,RandomOverSampler,NaN,NaN
freq,10,10,NaN,NaN
mean,NaN,NaN,0.075000,0.053933
std,NaN,NaN,0.025318,0.032535
min,NaN,NaN,0.050000,0.007539
25%,NaN,NaN,0.050000,0.030263
50%,NaN,NaN,0.075000,0.043478
75%,NaN,NaN,0.100000,0.066667


In [16]:
# Grouper les résultats par modèle
grouped_results = results_df.groupby('Model')

# Initialiser un DataFrame pour stocker les meilleurs résultats par modèle
best_results_df = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque groupe
for model, group_df in grouped_results:
    # Trouver la ligne avec le plus grand F1 score pour chaque modèle
    best_result = group_df.sort_values(by='F1 Score', ascending=False).iloc[0]
    
    # Ajouter le meilleur résultat au DataFrame final
    # Remplacez la ligne best_results_df = best_results_df.append(...) par ceci
    best_results_df = pd.concat([best_results_df, pd.DataFrame({
        'Model': [best_result['Model']],
        'Sampler': [best_result['Sampler']],
        'Sampling Strategy': [best_result['Sampling Strategy']],
        'F1 Score': [best_result['F1 Score']]
    })], ignore_index=True)

# Afficher le tableau récapitulatif des meilleurs résultats pour chaque modèle
print(best_results_df)
best_results_df.to_pickle('best_results_df.pkl')

                    Model             Sampler  Sampling Strategy  F1 Score
0      AdaBoostClassifier   RandomOverSampler               0.10  0.131428
1  DecisionTreeClassifier               SMOTE               0.05  0.037149
2      LogisticRegression   RandomOverSampler               0.10  0.106085
3  RandomForestClassifier  RandomUnderSampler               0.05  0.102525
4           XGBClassifier               SMOTE               0.10  0.077850


C:\Users\Admin\AppData\Local\Temp\ipykernel_27400\346220120.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  best_results_df = pd.concat([best_results_df, pd.DataFrame({


In [31]:
best_results_df.head(10)

,Model,Sampler,Sampling Strategy,F1 Score
0,AdaBoostClassifier,RandomOverSampler,0.10,0.131428
1,DecisionTreeClassifier,SMOTE,0.05,0.037149
2,LogisticRegression,RandomOverSampler,0.10,0.106085
3,RandomForestClassifier,RandomUnderSampler,0.05,0.102525
4,XGBClassifier,SMOTE,0.10,0.077850


## Maintenant qu'on a une vue d'ensemble claire sur les méthodes de re-échantillonage qui marchent le mieux en fonction des modèles, on va reprendre ces combinaisons pour réaliser une optimisation cette fois-ci des hyper-paramètres des modèles à l'aide d'un greedsearch.

### Model: GradientBoostingClassifier - Sampler: RandomUnderSampler - Sampling Strategy: 0.1

In [17]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import pickle
from imblearn.over_sampling import RandomOverSampler
from tqdm import tqdm

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.10)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle de l'AdaBoostClassifier
adab_classifier = AdaBoostClassifier(DecisionTreeClassifier()) # Utilise un arbre de décision faible par défaut

# Pipeline
pipeline = Pipeline([
    ('classifier', adab_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50],
    'classifier__estimator__max_depth': [1, 2],  # Ajustez ici pour le paramètre de l'arbre faible
    'classifier__learning_rate': [1],
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)

# Progress bar for grid search
for train_index, test_index in tqdm(tscv.split(X_train, y_train), desc="Grid Search Progress"):
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    grid_search.fit(X_train_fold, y_train_fold)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres (Adaboost):", grid_search.best_params_)
print("Meilleur score F1 (Adaboost):", grid_search.best_score_)

# Enregistrement du modèle
with open('modele_adaboost_newparam.pkl', 'wb') as file:
    pickle.dump(grid_search, file)

Grid Search Progress: 0it [00:00, ?it/s]C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. 

Meilleurs paramètres (Adaboost): {'classifier__estimator__max_depth': 2, 'classifier__learning_rate': 1, 'classifier__n_estimators': 50}
Meilleur score F1 (Adaboost): 0.07971802099148495


In [17]:
from sklearn.ensemble import RandomForestClassifier

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.05)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle du RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Pipeline
pipeline_rf = Pipeline([
    ('classifier', rf_classifier)
])

tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid_rf = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [None, 10, 20],  # Ajustez ici pour le paramètre de profondeur
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

# Recherche sur la grille
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, scoring='f1', cv=tscv)
grid_search_rf.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres (Random Forest):", grid_search_rf.best_params_)
print("Meilleur score F1 (Random Forest):", grid_search_rf.best_score_)

with open('modele_random_forest.pkl', 'wb') as file_rf:
    pickle.dump(grid_search_rf, file_rf)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
import pickle

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.10)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle du XGBClassifier
xgb_classifier = XGBClassifier()

# Pipeline
pipeline_xgb = Pipeline([
    ('classifier', xgb_classifier)
])

tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid_xgb = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [3, 7],  # Ajustez ici pour le paramètre de profondeur
    'classifier__learning_rate': [0.1, 0.01],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0],
}

# Recherche sur la grille
grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, scoring='f1', cv=tscv)
grid_search_xgb.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres (XGBoost):", grid_search_xgb.best_params_)
print("Meilleur score F1 (XGBoost):", grid_search_xgb.best_score_)

# Afficher les résultats détaillés de la recherche sur la grille
results = grid_search_xgb.cv_results_
for mean_score, params in zip(results["mean_test_score"], results["params"]):
    print(f"Score moyen: {mean_score}, Paramètres: {params}")

# Enregistrement du modèle
with open('modele_xgboost.pkl', 'wb') as file_xgb:
    pickle.dump(grid_search_xgb, file_xgb)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.1)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train, y_train)

 
# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

# Enregistrement du modèle
with open('modele_gboost.pkl', 'wb') as file_gb:
    pickle.dump(gb_classifier, file_gb)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.1)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train_ros, y_train_ros)

 
# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

In [ ]:
import numpy as np

# Compter le nombre d'éléments uniques dans y_train_rus
unique_classes, counts = np.unique(y_train_rus, return_counts=True)

# Afficher le nombre d'éléments uniques et leurs occurrences
for cls, count in zip(unique_classes, counts):
    print(f"Classe {cls}: {count} occurrences")

In [ ]:
len(y_train)

In [ ]:
len(y_train_rus)

In [ ]:
with open('modele_regression_logistique.pkl', 'rb') as file:
    modele_regression_logistique = pickle.load(file)

with open('modele_xgboost.pkl', 'rb') as file:
    modele_xgboost = pickle.load(file)

with open('modele_adaboost.pkl', 'rb') as file:
    modele_adaboost = pickle.load(file)

with open('modele_arbre_decision.pkl', 'rb') as file:
    modele_arbre_decision = pickle.load(file)

modeles = [modele_regression_logistique, modele_xgboost, modele_adaboost]

for model in modeles:
    # Faire des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Calculer le F1-score sur l'ensemble de test
    f1_test = f1_score(y_test, y_pred)

    # Afficher le F1-score sur l'ensemble de test et le nom du modèle
    print(f"F1 Score sur l'ensemble de test ({model.__class__.__name__}): {f1_test}")


In [38]:
with open('modele_adaboost_newparam.pkl', 'rb') as file:
    modele_adaboost = pickle.load(file)
    
modeles = [modele_adaboost]

for model in modeles:
    # Faire des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Calculer le F1-score sur l'ensemble de test
    f1_test = f1_score(y_test, y_pred)

    # Afficher le F1-score sur l'ensemble de test et le nom du modèle
    print(f"F1 Score sur l'ensemble de test ({model.__class__.__name__}): {f1_test}")

F1 Score sur l'ensemble de test (GridSearchCV): 0.05073746312684366
